In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
y_train = train.SalePrice
x_train = train.drop(columns = "SalePrice")

In [5]:
x_train.shape

(1460, 80)

In [6]:
test.shape

(1459, 80)

In [7]:
full_data = pd.concat([x_train, test])

In [8]:
full_data.shape

(2919, 80)

In [9]:
full_data.shape

(2919, 80)

In [10]:
#train.head()

In [11]:
full_data.dtypes.value_counts()

object     43
int64      26
float64    11
Name: count, dtype: int64

In [12]:
full_data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [13]:
null_columns = full_data.columns[full_data.isnull().any()]
print(null_columns)

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType'],
      dtype='object')


In [14]:
#null_counts = train[null_columns].isnull().sum()
#print(f" {null_counts}, {train[null_columns].dtypes} ")

In [15]:
full_data[null_columns].nunique()

MSZoning           5
LotFrontage      128
Alley              2
Utilities          2
Exterior1st       15
Exterior2nd       16
MasVnrType         3
MasVnrArea       444
BsmtQual           4
BsmtCond           4
BsmtExposure       4
BsmtFinType1       6
BsmtFinSF1       991
BsmtFinType2       6
BsmtFinSF2       272
BsmtUnfSF       1135
TotalBsmtSF     1058
Electrical         5
BsmtFullBath       4
BsmtHalfBath       3
KitchenQual        4
Functional         7
FireplaceQu        5
GarageType         6
GarageYrBlt      103
GarageFinish       3
GarageCars         6
GarageArea       603
GarageQual         5
GarageCond         5
PoolQC             3
Fence              4
MiscFeature        4
SaleType           9
dtype: int64

In [16]:
full_data[null_columns].dtypes

MSZoning         object
LotFrontage     float64
Alley            object
Utilities        object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinSF1      float64
BsmtFinType2     object
BsmtFinSF2      float64
BsmtUnfSF       float64
TotalBsmtSF     float64
Electrical       object
BsmtFullBath    float64
BsmtHalfBath    float64
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageCars      float64
GarageArea      float64
GarageQual       object
GarageCond       object
PoolQC           object
Fence            object
MiscFeature      object
SaleType         object
dtype: object

In [17]:
for column in full_data.columns:
    if full_data[column].dtype == 'float64' and full_data[column].isnull().any():
        mode_value = full_data[column].mode()[0]  
        full_data[column].fillna(mode_value, inplace=True)

/tmp/ipykernel_18/2617287886.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data[column].fillna(mode_value, inplace=True)


In [18]:
null_columns = full_data.columns[full_data.isnull().any()]
print(null_columns)

Index(['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PoolQC', 'Fence', 'MiscFeature', 'SaleType'],
      dtype='object')


In [19]:
null_counts = full_data[null_columns].isnull().sum()
print(f" {null_counts}, {full_data[null_columns].dtypes} ")

 MSZoning           4
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType      1766
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Electrical         1
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageFinish     159
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64, MSZoning        object
Alley           object
Utilities       object
Exterior1st     object
Exterior2nd     object
MasVnrType      object
BsmtQual        object
BsmtCond        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
Electrical      object
KitchenQual     object
Functional      object
FireplaceQu     object
GarageType      object
GarageFinish    object
GarageQual      object
GarageCond      object
PoolQC          object
Fence           object
MiscFeature     obj

In [20]:
full_data[null_columns]

,MSZoning,Alley,Utilities,Exterior1st,Exterior2nd,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,...,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature,SaleType
0,RL,NaN,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,No,GLQ,...,Typ,NaN,Attchd,RFn,TA,TA,NaN,NaN,NaN,WD
1,RL,NaN,AllPub,MetalSd,MetalSd,NaN,Gd,TA,Gd,ALQ,...,Typ,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN,WD
2,RL,NaN,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,Mn,GLQ,...,Typ,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN,WD
3,RL,NaN,AllPub,Wd Sdng,Wd Shng,NaN,TA,Gd,No,ALQ,...,Typ,Gd,Detchd,Unf,TA,TA,NaN,NaN,NaN,WD
4,RL,NaN,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,Av,GLQ,...,Typ,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,NaN,AllPub,CemntBd,CmentBd,NaN,TA,TA,No,Unf,...,Typ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1455,RM,NaN,AllPub,CemntBd,CmentBd,NaN,TA,TA,No,Rec,...,Typ,NaN,CarPort,Unf,TA,TA,NaN,NaN,NaN,WD
1456,RL,NaN,AllPub,VinylSd,VinylSd,NaN,TA,TA,No,ALQ,...,Typ,TA,Detchd,Unf,TA,TA,NaN,NaN,NaN,WD
1457,RL,NaN,AllPub,HdBoard,Wd Shng,NaN,Gd,TA,Av,GLQ,...,Typ,NaN,NaN,NaN,NaN,NaN,NaN,MnPrv,Shed,WD


In [21]:
unique_values = {}
for column in null_columns:
    unique_values[column] = full_data[column].unique()


In [22]:
print(unique_values)

{'MSZoning': array(['RL', 'RM', 'C (all)', 'FV', 'RH', nan], dtype=object), 'Alley': array([nan, 'Grvl', 'Pave'], dtype=object), 'Utilities': array(['AllPub', 'NoSeWa', nan], dtype=object), 'Exterior1st': array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock', nan], dtype=object), 'Exterior2nd': array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock', nan], dtype=object), 'MasVnrType': array(['BrkFace', nan, 'Stone', 'BrkCmn'], dtype=object), 'BsmtQual': array(['Gd', 'TA', 'Ex', nan, 'Fa'], dtype=object), 'BsmtCond': array(['TA', 'Gd', nan, 'Fa', 'Po'], dtype=object), 'BsmtExposure': array(['No', 'Gd', 'Mn', 'Av', nan], dtype=object), 'BsmtFinType1': array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', nan, 'LwQ'], dtype=object), 'BsmtFinTyp

In [23]:
unique_value_counts = {}
null_value_counts = {}

for column in null_columns:
    unique_value_counts[column] = full_data[column].value_counts()
    null_value_counts[column] = full_data[column].isnull().sum()

In [24]:
for column in null_columns:
    print(f"Column: {column}")
    print(f"Unique values:\n{full_data[column].value_counts()}\n")
    print(f"Null values count: {full_data[column].isnull().sum()}\n")

Column: MSZoning
Unique values:
MSZoning
RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: count, dtype: int64

Null values count: 4

Column: Alley
Unique values:
Alley
Grvl    120
Pave     78
Name: count, dtype: int64

Null values count: 2721

Column: Utilities
Unique values:
Utilities
AllPub    2916
NoSeWa       1
Name: count, dtype: int64

Null values count: 2

Column: Exterior1st
Unique values:
Exterior1st
VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
AsphShn       2
Stone         2
CBlock        2
ImStucc       1
Name: count, dtype: int64

Null values count: 1

Column: Exterior2nd
Unique values:
Exterior2nd
VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
BrkFace      47
Stucco       47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphSh

In [25]:
for column in null_columns:
    print(f"Value counts for column {column}:\n{full_data[column].value_counts()}\n")

Value counts for column MSZoning:
MSZoning
RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: count, dtype: int64

Value counts for column Alley:
Alley
Grvl    120
Pave     78
Name: count, dtype: int64

Value counts for column Utilities:
Utilities
AllPub    2916
NoSeWa       1
Name: count, dtype: int64

Value counts for column Exterior1st:
Exterior1st
VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
AsphShn       2
Stone         2
CBlock        2
ImStucc       1
Name: count, dtype: int64

Value counts for column Exterior2nd:
Exterior2nd
VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
BrkFace      47
Stucco       47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: count, dtype: int64

Value counts

In [26]:
for column in full_data.columns:
    if(full_data[column].isnull().sum()<160):
        mode_value = full_data[column].mode()[0]  
        full_data[column].fillna(mode_value, inplace=True) 
        
    

/tmp/ipykernel_18/1526273348.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data[column].fillna(mode_value, inplace=True)
/tmp/ipykernel_18/1526273348.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [27]:
null_columns = full_data.columns[full_data.isnull().any()]
print(null_columns)

Index(['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


In [28]:
null_counts = full_data[null_columns].isnull().sum()
print(f" {null_counts}, {full_data[null_columns].dtypes} ")

 Alley          2721
MasVnrType     1766
FireplaceQu    1420
PoolQC         2909
Fence          2348
MiscFeature    2814
dtype: int64, Alley          object
MasVnrType     object
FireplaceQu    object
PoolQC         object
Fence          object
MiscFeature    object
dtype: object 


In [29]:
columns_to_drop = ['Alley', 'MasVnrType','FireplaceQu','PoolQC','Fence']

In [30]:
full_data.drop(columns_to_drop, axis = 1, inplace = True )

In [31]:
null_columns = full_data.columns[full_data.isnull().any()]
print(null_columns)

Index(['MiscFeature'], dtype='object')


In [32]:
full_data.drop('MiscFeature', axis =1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,61,0,0,0,0,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,42,0,0,0,0,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,...,32,0,0,0,0,700,7,2006,WD,Normal


In [33]:
encoded_full_data = pd.get_dummies(full_data)


In [34]:
encoded_full_data.shape

(2919, 271)

In [35]:
train.shape

(1460, 81)

In [36]:
test.shape

(1459, 80)

In [37]:
x_train = encoded_full_data[:1460]

In [38]:
x_train.shape

(1460, 271)

In [39]:
x_test = encoded_full_data[1460:]

In [40]:
x_test.shape

(1459, 271)

In [41]:
y_train.shape

(1460,)

In [42]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
# Instantiate the model
model = XGBRegressor()

# Fit the model to the training data
model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [43]:
y_pred = model.predict(x_test)

In [44]:
y_pred

array([122893.78, 152178.  , 182046.48, ..., 154871.86, 105466.15,
       207151.34], dtype=float32)

In [45]:
sample_submission_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_submission_df['SalePrice'] = y_pred
sample_submission_df.to_csv('/kaggle/working/submission.csv', index=False)
sample_submission_df.head()

,Id,SalePrice
0,1461,122893.781250
1,1462,152178.000000
2,1463,182046.484375
3,1464,185564.265625
4,1465,198350.312500
